# JPAD test within Jupyter running a BeakerX Java kernel

— Agostino De Marco, University of Naples Federico II

JPAD stands for *Java Program toolchain for Aircraft Designers*. JPAD is a Java software library containing classes and utility functions that can be used to build software systems. Typically, these are calculation programs providing aerodynamic and performance predictions for given aircraft designs.

<img src="./images/IRONa.png" width="60%" />

JPAD is open source and its [public repository is on GitHub](https://github.com/Aircraft-Design-UniNa/jpad).


This Jupyter notebook uses the [BeakerX Java kernel](http://beakerx.com). The purpose is demonstrating:

- the use of Java with Jupyter
- the use of JPAD and its companion external native libraries in the context of a notebook

## Running this notebook

Let us print the current directory content. We are Java users, so let start using Java in our first code cell to this purpose:

In [1]:
import java.nio.file.*;
Files.list(Paths.get("."))
    .forEach(System.out::println);

.\.ipynb_checkpoints
.\;
.\beakerx-jpad.bat
.\images
.\jar
.\lib
.\Test_JPAD_1.ipynb


null

This notebook is named `Test_JPAD_1.ipynb`. The `jar` subdirectory contains the two Jar files

- `JPADConfigs_31032018.jar`
- `JPADCore_v2_31032018.jar`

where JPAD is bundled, and all the Jar's that JPAD depends upon.

In addition, the The `lib` subdirectory contains all the native libraries (Win64) needed for JPAD to work correctly.

The BeakerX kernel for this notebook is launched on Windows with the following `beakerx-jpad.bat` file:

```
setlocal
set PATH=%PATH%;.\lib;.\jar;
set JUPYTER_RUNTIME_DIR=%JUPYTER_RUNTIME_DIR%;.\lib
set JUPYTER_EXEC="beakerx"

%JUPYTER_EXEC%
```

It adds `lib` and `jar` directories to the `PATH` environment variables as well as to the `JUPYTER_RUNTIME_DIR`. Then, it runs Jupyter with the `beakerx` command. Once Jupyter with the [BeakerX](http://beakerx.com/) kernel is set up and running, the notebook file can be opened and executed.

## Preliminaries

### Classpath extras

The following cell uses BeakerX *magic command* `%classpath` that directs the Java kernel to include all the Jar files located in the path `./jar`. The command `%classpath` alone prints out all the included Jars.

In [2]:
%classpath add jar .\jar\*
%classpath

Added jars: [xstream-1.4.7.jar, jarh4obj.jar, jxls-core-1.0.5.jar, jarh5obj.jar, flanagan.jar, dom4j-1.6.1.jar, log4j-1.2.13.jar, xmlbeans-2.6.0.jar, JPADConfigs_31032018.jar, processing-core.jar, freebuilder-1.10.5.jar, commons-codec-1.5.jar, jfreesvg-3.0.jar, MOEAFramework-2.12.jar, jdom-2.0.5.jar, unit-api-0.6.1.jar, commons-lang3-3.7.jar, jarhdf-2.10.1.jar, vecmath.jar, trove-3.1a1.jar, commons-logging-1.1.jar, slf4j-api-1.7.5.jar, jfreechart-1.5.0.jar, jarhdfobj.jar, jxls-reader-1.0.5.jar, orsonpdf-1.7.jar, fxgraphics2d-1.3.jar, commons-logging-1.2.jar, jfreesvg-3.3.jar, commons-math3-3.6.1.jar, log4j-core-2.0.2.jar, commons-jexl-2.1.1.jar, reflectutils-0.9.16.jar, jarhdf5-2.10.1.jar, jfreechart-nofx-2.0-pre1.jar, commons-collections4-4.0.jar, javaslang-2.0.4.jar, gtmat.jar, orsoncharts-1.5.jar, poi-3.10.1-20140818.jar, junit-4.11.jar, JPADCore_v2_31032018.jar, tigl-2.2.1.jar, stax-api-1.0.1.jar, jscience.jar, log4j-api-2.0.2.jar, guava-16.0.1.jar, poi-ooxml-3.10.1-20140818.jar, i

### Check the native libs

The native libs stored in the `./lib` directory are listed by the following code cell.

In [3]:
import java.nio.file.*;
Files.list(Paths.get("./lib")) // a stream of paths
    .map(path -> path.toString().toLowerCase())
    .filter(name -> name.matches("([^\\s]+(\\.(?i)(lib|dll))$)")) // use regex
    .forEach(System.out::println);

.\lib\jhdf.dll
.\lib\jhdf.lib
.\lib\jhdf5.dll
.\lib\jhdf5.lib
.\lib\libhdf.lib
.\lib\libhdf5.lib
.\lib\libjpeg.lib
.\lib\libmfhdf.lib
.\lib\libszip.lib
.\lib\libxdr.lib
.\lib\libzlib.lib
.\lib\msvcp100.dll
.\lib\msvcr100.dll
.\lib\tbb.dll
.\lib\tbbmalloc.dll
.\lib\tigl.dll
.\lib\tigl.lib
.\lib\tixi.dll
.\lib\vcomp100.dll


null

## Calling JPAD functions

At this point we can start using the JPAD API functions.

### Global scope imports

The following are BeakerX *magics*, a bunch of `%import` commands that make a set of Java packages globally visible by all the subsequent code cells.

In [4]:
%import java.io.File
%import java.io.IOException
%import java.io.OutputStream
%import java.io.PrintStream

%import java.nio.file.Paths
%import java.nio.file.Files

%import java.util.ArrayList
%import java.util.List

%import javax.measure.quantity.*
%import javax.measure.unit.SI

%import org.jscience.physics.amount.Amount

%import org.apache.poi.openxml4j.exceptions.InvalidFormatException

%import ncsa.hdf.hdf5lib.exceptions.HDF5LibraryException

%import configuration.*
%import configuration.enumerations.*

%import aircraft.*
%import aircraft.components.*

%import aircraft.components.FuelTank

%import aircraft.components.fuselage.*
%import aircraft.components.liftingSurface.*
%import aircraft.components.liftingSurface.creator.*
%import aircraft.components.nacelles.*
%import aircraft.components.powerplant.*

// TODO: analyses
// TODO: cad

%import calculators.*
%import calculators.aerodynamics.*
%import calculators.balance.*
%import calculators.costs.*
%import calculators.geometry.*
%import calculators.performance.*
%import calculators.stability.*
%import calculators.weights.*

%import database.*
%import database.databasefunctions.*
%import database.databasefunctions.aerodynamics.*
%import database.databasefunctions.aerodynamics.fusDes.*
%import database.databasefunctions.aerodynamics.vedsc.*
%import database.databasefunctions.engine.*

%import standaloneutils.*
%import standaloneutils.atmosphere.*
%import standaloneutils.cpacs.*
%import standaloneutils.customdata.*
%import standaloneutils.database.*
%import standaloneutils.jsbsim.*
%import standaloneutils.launchers.*
%import standaloneutils.mathtools.*
%import standaloneutils.aerotools.*

%import writers.*


### Setup database files and create an Aircraft object

We will read the input file `../in/aircraft_IRON_NEW.xml` that defines a specific aircraft design named *IRON*. The following image is a top view of this airplane.

<img src="./images/IRONc.png" width="60%" />

To do so we create a package named `test.jpad` that wraps a test class named `JPADTest`. Objects of this class incorporate the paths to all the files necessary for JPAD to work correctly.

In [5]:
package test.jpad;

public class JPADTest {
    
    public String pathToXML;
    public String aircraftFileName;
    
    public String dirAirfoil;
    public String dirFuselages;
    public String dirLiftingSurfaces;
    public String dirEngines;
    public String dirNacelles;
    public String dirLandingGears;
    public String dirCabinConfiguration;
    
    public String databaseFolderPath;
    public String inputFolderPath;
    public String outputFolderPath;

    public AerodynamicDatabaseReader aeroDatabaseReader;
    public HighLiftDatabaseReader highLiftDatabaseReader;
    public FusDesDatabaseReader fusDesDatabaseReader;
    public VeDSCDatabaseReader veDSCDatabaseReader;

    public JPADTest(String fileName) {
        
        aircraftFileName = fileName;
        
        System.out.println("--------------------------");
        System.out.println("Initialize JPADTest");
        System.out.println("--------------------------");
        
        MyConfiguration.setDir(
            FoldersEnum.DATABASE_DIR,
            Paths.get("../data").toString()
        );
        MyConfiguration.setDir(
            FoldersEnum.INPUT_DIR,
            Paths.get("../in").toString()
        );

        MyConfiguration.setDir(
            FoldersEnum.OUTPUT_DIR,
            Paths.get("../out").toString()
        );

        String aerodynamicDatabaseFileName = "Aerodynamic_Database_Ultimate.h5";
        String highLiftDatabaseFileName = "HighLiftDatabase.h5";
        String fusDesDatabaseFilename = "FusDes_database.h5";
        String vedscDatabaseFilename = "VeDSC_database.h5";

        databaseFolderPath = MyConfiguration.getDir(FoldersEnum.DATABASE_DIR);
        inputFolderPath = MyConfiguration.getDir(FoldersEnum.INPUT_DIR);
        outputFolderPath = MyConfiguration.getDir(FoldersEnum.OUTPUT_DIR);

        System.out.println("Database dir: " + databaseFolderPath);
        System.out.println("Input dir: " + inputFolderPath);
        System.out.println("Output dir: " + outputFolderPath);

        aeroDatabaseReader = DatabaseManager.initializeAeroDatabase(
            new AerodynamicDatabaseReader(
                databaseFolderPath,
                aerodynamicDatabaseFileName
                ),
            databaseFolderPath
            );

        highLiftDatabaseReader = DatabaseManager.initializeHighLiftDatabase(
            new HighLiftDatabaseReader(
                databaseFolderPath,
                highLiftDatabaseFileName),
            databaseFolderPath
            );

        fusDesDatabaseReader = DatabaseManager.initializeFusDes(
            new FusDesDatabaseReader(
                databaseFolderPath,
                fusDesDatabaseFilename),
            databaseFolderPath
            );

        veDSCDatabaseReader = DatabaseManager.initializeVeDSC(
            new VeDSCDatabaseReader(
                databaseFolderPath,
                vedscDatabaseFilename),
            databaseFolderPath
            );

        pathToXML = inputFolderPath + "/" + aircraftFileName;
        dirAirfoil = inputFolderPath + "/lifting_surfaces/airfoils";
        dirFuselages = inputFolderPath + "/fuselages";
        dirLiftingSurfaces = inputFolderPath + "/lifting_surfaces";
        dirEngines = inputFolderPath + "/engines";
        dirNacelles = inputFolderPath + "/nacelles";
        dirLandingGears = inputFolderPath + "/landing_gears";
        dirCabinConfiguration = inputFolderPath + "/cabin_configurations";
        // atmosphere = AtmosphereCalc.getAtmosphere(1000.);

        System.out.println("--------------------------");
        System.out.println("Initialization done");
        System.out.println("--------------------------");
        
    }
}

test.jpad.JPADTest

For the sake of simplicity, the class `JPADTest` has public member variables. Some of them are of types provided by the JPAD API. The class constructor `JPADTest()` defines the paths of all auxiliary input files and directories and takes as an input variable the name of the aircraft definition file (a file in XML format).

The following code cell defines an object `test` of type `JPADTest`, that incorporates all the necessary definitions to read the IRON aircraft design. The object `test` is initialized by passing the main aircraft definition file stored `../in/aircraft_IRON_NEW.xml`

Successively, an object `aircraft` of type `Aircraft` is created and printed using the `Aircraft.toString()` function.

In [6]:
package test.jpad;

JPADTest test = new JPADTest("aircraft_IRON_NEW.xml");

try {
    
    System.out.println("--------------------------");
    System.out.println("Aircraft import");
    System.out.println("--------------------------");
    
    Aircraft theAircraft = Aircraft.importFromXML(
        test.pathToXML,
        test.dirLiftingSurfaces,
        test.dirFuselages,
        test.dirEngines,
        test.dirNacelles,
        test.dirLandingGears,
        test.dirCabinConfiguration,
        test.dirAirfoil,
        test.aeroDatabaseReader,
        test.highLiftDatabaseReader,
        test.fusDesDatabaseReader,
        test.veDSCDatabaseReader
        );

    // print out the object
    System.out.println(theAircraft.toString());
    
} catch (Throwable t) {
    t.printStackTrace();
    throw t;
}

--------------------------
Initialize JPADTest
--------------------------
Database dir: ..\data
Input dir: ..\in
Output dir: ..\out
De-serializing file: C:\Users\agodemar\Dropbox\Jupyter_2\beakerx\jpad\test2\..\data\serializedDatabase\interpolaterAerodynamicDatabase.xml ...
database.databasefunctions.aerodynamics.AerodynamicDatabaseReader de-serialization complete
De-serializing file: C:\Users\agodemar\Dropbox\Jupyter_2\beakerx\jpad\test2\..\data\serializedDatabase\interpolaterHighLiftDatabase.xml ...
database.databasefunctions.aerodynamics.HighLiftDatabaseReader de-serialization complete
De-serializing file: C:\Users\agodemar\Dropbox\Jupyter_2\beakerx\jpad\test2\..\data\serializedDatabase\interpolaterFusDesDatabase.xml ...
database.databasefunctions.aerodynamics.fusDes.FusDesDatabaseReader de-serialization complete
De-serializing file: C:\Users\agodemar\Dropbox\Jupyter_2\beakerx\jpad\test2\..\data\serializedDatabase\interpolaterVeDSCDatabase.xml ...
database.databasefunctions.aerodyna

[0.0 m, 5.7799999999999958 m]
zLE Break-Points ->
[0.0 m, 0.0 m]
Chords Break-Points ->
[5.16 m, 3.3099999999999996 m]
Twists Break-Points ->
[0.0 deg, 0.0 deg]
[LiftingSurfaceCreator] Map _panels to spanwise discretized Ys ...
[LiftingSurfaceCreator] Map _panels to spanwise discretized chords ...
[LiftingSurfaceCreator] Map _panels to spanwise discretized Xle, Yle, twist ...
[LiftingSurfaceCreator] Map Ys to spanwise discretized variables ...
File C:\Users\agodemar\Dropbox\Jupyter_2\beakerx\jpad\test2\..\in\lifting_surfaces\canard_IRON_v2.xml found.
Parsing ...
File ..\in/lifting_surfaces\canard_IRON_v2.xml parsed.
Reading lifting surface data ...
Panels found: 1
[0]
Panel id: Canard panel
Reading lifting surface panel data from XML doc ...
File C:\Users\agodemar\Dropbox\Jupyter_2\beakerx\jpad\test2\..\in\lifting_surfaces\airfoils\CANARD_IRON.xml found.
Parsing ...
File ..\in/lifting_surfaces/airfoils\CANARD_IRON.xml parsed.
Reading airfoil data ...
File C:\Users\agodemar\Dropbox\Jupy

	Outline YZ Upper Section View - Z (m): [0.0, -0.05947885662317276, -0.15624375641345978, -0.2809745669364929, -0.42520442605018616, -0.5813197493553162, -0.7425600290298462, -0.903018057346344, -1.0576401948928833, -1.2022255659103394, -1.3334267139434814, -1.4487498998641968, -1.5465539693832397, -1.6260515451431274, -1.6873079538345337, -1.7312424182891846, -1.7596266269683838, -1.7750864028930664, -1.7811001539230347, -1.781999945640564, -1.781999945640564, -1.7811003923416138, -1.7750862836837769, -1.7596266269683838, -1.731242060661316, -1.6873080730438232, -1.6260517835617065, -1.5465539693832397, -1.4487497806549072, -1.3334267139434814, -1.2022253274917603, -1.0576400756835938, -0.903018057346344, -0.7425600290298462, -0.5813199281692505, -0.42520472407341003, -0.2809748947620392, -0.15624406933784485, -0.05947909504175186, -1.2745884703235788E-7]
	...............................................................................................................
	-----------------

	Cl curve (deg) = [-2.5702408727542947, -2.436761653458201, -2.3032824341621083, -2.1698032148660147, -2.036323995569921, -1.9028447762738272, -1.7693655569777336, -1.6358863376816404, -1.5024071183855467, -1.3689278990894531, -1.2354486797933597, -1.101969460497266, -0.9684902412011722, -0.8350110219050788, -0.7015318026089847, -0.5680525833128913, -0.43457336401679725, -0.30109414472070384, -0.1676149254246101, -0.03413570612851635, 0.09934351316757739, 0.23282273246367108, 0.36630195175976477, 0.4997811710558584, 0.633260390351952, 0.7667396096480458, 0.9002188289441394, 1.0336980482402331, 1.1671772675363268, 1.3006564868324204, 1.4341357061285143, 1.567614925424608, 1.683890877087232, 1.7668764103367893, 1.8219941751080708, 1.8546882852335598, 1.8704028545457367, 1.874581996877085, 1.8726698260600845, 1.8701104559272181, 1.8723480003109678, 1.88482657304381, 1.912990287958233, 1.9622832588867158, 2.0381495996617387, 2.146033424115786, 2.2913788460813382, 2.4796299793908716, 2.7162

	x coordinates = [1.0, 0.994006, 0.983386, 0.970838, 0.956749, 0.941664, 0.926069, 0.910206, 0.894206, 0.878188, 0.862181, 0.846135, 0.830169, 0.814353, 0.798675, 0.783203, 0.767887, 0.75264, 0.737487, 0.722361, 0.707239, 0.692153, 0.677029, 0.661927, 0.646819, 0.631622, 0.61641, 0.601129, 0.585763, 0.570377, 0.554889, 0.539335, 0.523758, 0.508071, 0.492338, 0.476598, 0.460783, 0.444983, 0.4292, 0.41334, 0.397526, 0.381718, 0.365856, 0.350067, 0.334254, 0.318437, 0.302698, 0.286939, 0.271235, 0.255626, 0.240016, 0.224544, 0.209135, 0.193824, 0.178699, 0.163656, 0.148894, 0.134303, 0.120093, 0.106264, 0.093014, 0.080466, 0.068817, 0.058277, 0.048968, 0.04095, 0.034107, 0.028318, 0.023422, 0.019268, 0.015742, 0.012736, 0.010163, 0.007973, 0.006131, 0.004588, 0.003305, 0.002258, 0.001429, 8.04E-4, 3.67E-4, 1.34E-4, 1.24E-4, 3.85E-4, 0.001067, 0.002323, 0.004054, 0.006076, 0.008322, 0.010813, 0.013638, 0.016901, 0.020701, 0.025156, 0.030445, 0.03683, 0.044552, 0.053784, 0.064671, 0.076984,

...

	Cl for Cd curve (deg) = [-3.0, -2.877551020408163, -2.7551020408163263, -2.6326530612244894, -2.5102040816326525, -2.3877551020408156, -2.2653061224489788, -2.142857142857142, -2.020408163265305, -1.8979591836734684, -1.7755102040816317, -1.653061224489795, -1.5306122448979584, -1.4081632653061218, -1.2857142857142851, -1.1632653061224485, -1.0408163265306118, -0.9183673469387751, -0.7959183673469383, -0.6734693877551016, -0.5510204081632648, -0.42857142857142805, -0.3061224489795913, -0.18367346938775456, -0.061224489795917825, 0.06122448979591891, 0.18367346938775564, 0.3061224489795924, 0.42857142857142916, 0.5510204081632659, 0.6734693877551027, 0.7959183673469394, 0.9183673469387762, 1.040816326530613, 1.1632653061224496, 1.2857142857142863, 1.408163265306123, 1.5306122448979596, 1.6530612244897962, 1.7755102040816328, 1.8979591836734695, 2.0204081632653064, 2.1428571428571432, 2.26530612244898, 2.387755102040817, 2.510204081632654, 2.6326530612244907, 2.7551020408163276, 2.87755

	Transition point lower side = 0.12

	.....................................
	                            panel tip
	c_t = 5.16 m
	epsilon_t = 0.0 deg
	-------------------------------------
	Airfoil
	-------------------------------------
	Name: NACA0012
	Type: CONVENTIONAL
	t/c = 0.12
	r_le/c = 0.0158
	x coordinates = [1.0, 0.99372, 0.982766, 0.969978, 0.955648, 0.940249, 0.924212, 0.907834, 0.89128, 0.874637, 0.857947, 0.84123, 0.824498, 0.807755, 0.791005, 0.77425, 0.757492, 0.740731, 0.72397, 0.707209, 0.690451, 0.673695, 0.656944, 0.640199, 0.62346, 0.606731, 0.590011, 0.573303, 0.556607, 0.539927, 0.523262, 0.506616, 0.48999, 0.473385, 0.456805, 0.440251, 0.423726, 0.407233, 0.390775, 0.374354, 0.357976, 0.341643, 0.325361, 0.309135, 0.292972, 0.276878, 0.260863, 0.244936, 0.22911, 0.213401, 0.197828, 0.182416, 0.167201, 0.152227, 0.137561, 0.123289, 0.109532, 0.096442, 0.084194, 0.072956, 0.06285, 0.053918, 0.046123, 0.039362, 0.033509, 0.028432, 0.024011, 0.020146, 0.016754, 0.01

	z coordinates = [-0.002003, -7.64E-4, 0.001585, 0.00449, 0.007778, 0.011251, 0.014783, 0.018313, 0.021838, 0.025327, 0.028755, 0.032114, 0.035408, 0.038631, 0.04178, 0.044851, 0.047834, 0.050723, 0.053511, 0.056194, 0.058768, 0.061229, 0.063572, 0.065794, 0.067894, 0.069866, 0.071695, 0.073371, 0.074887, 0.076226, 0.0774, 0.078418, 0.079284, 0.080009, 0.080607, 0.081083, 0.081439, 0.081675, 0.081785, 0.081777, 0.081646, 0.081393, 0.081026, 0.080543, 0.079943, 0.079228, 0.078394, 0.077439, 0.076365, 0.07516, 0.073823, 0.072354, 0.07074, 0.06898, 0.067054, 0.064952, 0.062664, 0.060171, 0.057472, 0.054549, 0.051417, 0.048078, 0.044567, 0.040946, 0.037298, 0.033653, 0.030086, 0.026683, 0.023577, 0.02081, 0.018328, 0.016059, 0.013936, 0.011916, 0.009968, 0.008063, 0.006192, 0.004353, 0.002548, 7.95E-4, -9.18E-4, -0.002657, -0.004416, -0.00615, -0.007813, -0.009377, -0.010811, -0.012083, -0.013212, -0.014273, -0.015321, -0.016413, -0.01762, -0.019061, -0.020829, -0.022873, -0.025043, -0.027

	1,810	1,584	0,319	0,000	0,000
	1,940	1,561	0,342	0,000	0,000
	2,069	1,538	0,365	0,000	0,000
	2,198	1,516	0,388	0,000	0,000
	2,328	1,493	0,410	0,000	0,000
	2,457	1,470	0,433	0,000	0,000
	2,586	1,447	0,456	0,000	0,000
	2,716	1,424	0,479	0,000	0,000
	2,845	1,402	0,502	0,000	0,000
	2,974	1,379	0,524	0,000	0,000
	3,103	1,356	0,547	0,000	0,000
	3,233	1,333	0,570	0,000	0,000
	3,362	1,310	0,593	0,000	0,000
	3,491	1,288	0,616	0,000	0,000
	3,621	1,265	0,638	0,000	0,000
	3,750	1,242	0,661	0,000	0,000
	Overall wing derived data
	Span: 7.499999999999999 m
	Semi-span: 3.75 m
	Surface of planform: 11.794687499999995 m²
	Surface wetted: 23.58937499999999 m²
	Aspect-ratio: 4.76909625625944
	Mean aerodynamic chord: 1.5957949401755238 m
	(X,Y,Z)_LE of mean aerodynamic chord: [0.3074439833783946 m, 1.7436014734847398 m, 0.0 m]
	Symmetric Flaps Control surface area: 3.6576899999999997 m²
	Slats Control surface area: 0.0 m²
	Asymmetric Flaps Control surface area: 0.0 m²
	Spoilers Control surface area: 0.0 

null

Have a look at the output of the above cell! Such a huge amount of data represents the geometry and the basic aerodynamic characteristics of the chosen aircraft design.